# Fortgerschrittenere Regex-Funktionen

Dieses Notebook behandelt komplexere Funktionen und Usecases für reguläre Ausdrücke in Python

## Capture Groups

Bisher haben wir ausschließlich mit dem kompletten Match des kompletten regulären Ausdrucks gearbeitet. Mithilfe von Capture Groups ist es aber möglich, direkt auf Teilbereiche des Matchs zuzugreifen, z. B. um gewisse Informationen aus einem längeren String zu extrahieren.

### Einfache Capture Groups

Gruppen in regulären Ausdrücken werden durch normale Klammern definiert und alle Gruppen sind in Python automatisch Capture Groups. Das heißt also, dass folgender regulärer Ausdruck eigentlich schon eine Capture Group beinhaltet: `(Dozent|Tutor)(in|innen)?`

In der Praxis heißt das, dass der gematchte Inhalt eines regulären Teilausdrucks, der in Klammern steht, im Match-Objekt immer separat abgerufen werden kann. Zugriff auf die einzelnen Teilmatches der Capture Groups geschieht über die bekannten Zugriffsoperationen auf Match-Objekte:
```python
mo[i]
mo.group(i)
```
`i` steht dabei für den Index der gewünschten Gruppe. Bisher haben wir nur den Index 0 für das komplette Match benutzt und die erste Capture Group erhält dann den Index 1 usw.

Alternativ kann auch mit `mo.groups()` auf alle Untergruppen eines Match-Objekts zugegriffen werden.

In [1]:
import re

regex = r"(Dozent|Tutor)(in|innen)?"

match = re.match(regex, "Dozentin")

print(match)

print(match[0])
print(match[1])
print(match[2])

print(match.groups())

<re.Match object; span=(0, 8), match='Dozentin'>
Dozentin
Dozent
in
('Dozent', 'in')


Capture Groups können dabei auch verschachtelt sein, wobei jede Gruppe einen eigenen Index bekommt und das Match von umschließenden Gruppen das Match der umschlossenen enthalten.

In [2]:
re.match(r"(\w(\w(\w(\w))))", "Test").groups()

('Test', 'est', 'st', 't')

Capture Groups sind vor allem dann nützlich, wenn das Muster eines Inputstrings wichtig ist, aber nur ein Teilstrings extrahiert werden sollen. Im folgenden Beispel wird z. B. aus Telefonbucheinträgen im CSV-Format nur der Name und die Telefonnummer der Personen extrahiert. 

In [3]:
phonebook_data = """
id,first_name,last_name,phone,address
1,Augustin,Heimes,7277895468,Shangsha
2,Morgen,Whitlock,9448781698,Reda
3,Antonia,Beddo,7234194863,Mikhaylovka
4,Lianne,Cassell,1677332412,Himi
5,Susy,O'Gorman,2521783032,Jinka
6,Corri,Gorvette,9581814951,Rendian
random line not matching our scheme
7,Aurelion,Sol,2016032424,Universe
8,Jessica,Goldring,2534678757,Sendang
9,Forbes,Splaven,3259725293,Sarimukti Kaler
10,Halsy,Howgate,6189139520,Sardoal
11,Michell,Layland,5566438363,Gbawe
"""

phonebook_regex = r"\d+,(\w+),(\w+),(\d+),\w+\n"

for match in re.finditer(phonebook_regex, phonebook_data):
    print(f"Die Telefonnummer von {match[1]} {match[2]} ist {match[3]}")

Die Telefonnummer von Augustin Heimes ist 7277895468
Die Telefonnummer von Morgen Whitlock ist 9448781698
Die Telefonnummer von Antonia Beddo ist 7234194863
Die Telefonnummer von Lianne Cassell ist 1677332412
Die Telefonnummer von Corri Gorvette ist 9581814951
Die Telefonnummer von Aurelion Sol ist 2016032424
Die Telefonnummer von Jessica Goldring ist 2534678757
Die Telefonnummer von Halsy Howgate ist 6189139520
Die Telefonnummer von Michell Layland ist 5566438363


### Non-Capturing Groups

Non-Capturing Groups sind Gruppen, die zwar gematcht werden, ihr Inhalt ist aber nicht wie bei Capture Groups separat abrufbar. Das heißt also, dass sie normal gematcht werden und im kompletten Match (also `group(0)`) auftauchen, aber nicht in `groups()`. Das ist z. B. nützlich um optionale Inhalte aus der Liste der Capture Groups fernzuhalten. Die Syntax für Non-Capturing Groups ist `(?:...)`, also eine normale Gruppe mit `?:` nach der öffnenden Klammer.

In [4]:
string = "this is the 1st test of non-capturing groups. we are currently in the 2nd jupyter notebook explaining more complex regex concepts. random numbers: 4 79 2873 2 1337 0"

english_numbers = re.compile(r"([0-9]+)(?:st|nd|rd|th)?")

for match in english_numbers.finditer(string):
    print(match.groups())

('1',)
('2',)
('4',)
('79',)
('2873',)
('2',)
('1337',)
('0',)


### Named Capture Groups;

Capture Groups können auch optional benannt werden. Das hat den Vorteil, dass auf das Teilmatch über einen Bezeichner anstatt eines Indexes zugegriffen werden kann. Named Capture Groups werden mit folgender Syntax angegeben: `(?P<name>...)`, wobei `name` eine beliebige Bezeichnung sein kann.

Zugriff auf diese Teilmatches erfolgt weiterhin über die bekannten Zugriffsoperationen, nur, dass anstatt eines numerischen Indexes der Bezeichner als String angegeben werden muss.
```python
mo["name"]
mo.group("name")
```

Außerdem ist es möglich, alle benannten Gruppen in einem Dictionary vom Match-Objekt zu bekommen:
```pyhon
mo.groupdict()
```

Wenn wir Named Capture Groups auf unser Telefonbuch-Beispiel anwenden, wird der Code deutlich lesbarer.

In [6]:
better_phonebook_regex = r"\d+,(?P<vorname>\w+),(?P<nachname>\w+),(?P<telefonnummer>\d+),\w+\n"

for match in re.finditer(better_phonebook_regex, phonebook_data):
    print(match.groupdict())
    print(f"Die Telefonnummer von {match['vorname']} {match['nachname']} ist {match['telefonnummer']}")

{'vorname': 'Augustin', 'nachname': 'Heimes', 'telefonnummer': '7277895468'}
Die Telefonnummer von Augustin Heimes ist 7277895468
{'vorname': 'Morgen', 'nachname': 'Whitlock', 'telefonnummer': '9448781698'}
Die Telefonnummer von Morgen Whitlock ist 9448781698
{'vorname': 'Antonia', 'nachname': 'Beddo', 'telefonnummer': '7234194863'}
Die Telefonnummer von Antonia Beddo ist 7234194863
{'vorname': 'Lianne', 'nachname': 'Cassell', 'telefonnummer': '1677332412'}
Die Telefonnummer von Lianne Cassell ist 1677332412
{'vorname': 'Corri', 'nachname': 'Gorvette', 'telefonnummer': '9581814951'}
Die Telefonnummer von Corri Gorvette ist 9581814951
{'vorname': 'Aurelion', 'nachname': 'Sol', 'telefonnummer': '2016032424'}
Die Telefonnummer von Aurelion Sol ist 2016032424
{'vorname': 'Jessica', 'nachname': 'Goldring', 'telefonnummer': '2534678757'}
Die Telefonnummer von Jessica Goldring ist 2534678757
{'vorname': 'Halsy', 'nachname': 'Howgate', 'telefonnummer': '6189139520'}
Die Telefonnummer von Halsy

### Back References

In regulären Ausdrücken mit Capture Groups kann mehrfach auf die selbe Capture Group verwiesen werden. Das nennt man eine Back Reference. Dabei wird nicht das Muster der Gruppe referenziert, sondern ihr Match. Für Back References gibt es zwei Schreibweisen:
- Für unbenannte Capture Groups wird per `\n` auf die `n`te Gruppe zugegriffen, wobei `n` für einen beliebigen numerischen Index steht. Z. B. `\1`
- Für den Zugriff auf benannte Gruppen gibt es die Syntax `(?P=name)`, wobei `name` für einen beliebigen Gruppenbezeichner steht. Z. B. `(?P=nachname)`

In [7]:
# wir wollen nur Wörter ausgeben, die mit dem selben Buchstaben beginnen und aufhören
words = ["Kiosk", "Sonnenschirm", "Megaphon", "Thermometer", "Bemerkungen", "Vergeblich", "Limette", "Heften", "Laryngitis", "Buckel", "Rocker", "Insel", "Warteschlange", "Route", "Gewehrfeuer", "Trockenheit", "Ochse", "Unerwartet", "Tiefe", "Prinzessin"]

regex = r"(\w)\w+\1"
#         ^ g1   ^ Back Reference 

[w for w in words if re.fullmatch(regex, w, re.IGNORECASE)]

['Kiosk', 'Rocker', 'Trockenheit']

### Zusätzliche Möglichkeiten in bekannten Methoden

#### re.sub()

Im Falle von `re.sub()` wissen wir schon, dass man im Ersetzungsstring die Back Reference `\g<0>` benutzen kann, um auf den kompletten zu ersetzenden Teilstring zu verweisen. Zudem ist es auch möglich, auf alle Capture Groups zu verweisen. Für Back References gibt es hier sogar drei Schreibweisen:
- `\n` für unbenannte Capture Groups, wobei `n` für einen beliebigen numerischen Index steht. Z. B. `\1`
- `\g<n>` für unbenannte Capture Groups. Diese Schreibweise vermeidet Doppeldeutigkeiten und ist vorzuziehen.
- `\g<name>` für benannte Capture Groups, wobei `name` für einen beliebigen Gruppenbezeichner steht. Z. B. `\g<nachname>`

In [8]:
# Ein paar Amerikaner haben uns Datumsangaben im Format MM/DD/YYYY gegeben. Wir wollen die Daten in die richtige Reihenfolge bringen und als DD.MM.YYYY ausgeben

dates = ["02/22/2020", "01/26/2019", "10/30/2019", "04/08/2019", "08/26/2019", "11/12/2018", "01/29/2019", "12/04/2018", "12/26/2019", "05/10/2019"]

# option 1, unbenannte Capture Groups
regex = r"(\d{2})/(\d{2})/(\d{4})"
repl  = r"\2.\1.\3" # Raw String, da \1 sonst als Escape Sequence angesehen wird 

# option 2, benannte Capture Groups
regex = r"(?P<month>\d{2})/(?P<day>\d{2})/(?P<year>\d{4})"
repl  = r"\g<day>.\g<month>.\g<year>"

[re.sub(regex, repl, d) for d in dates]

['22.02.2020',
 '26.01.2019',
 '30.10.2019',
 '08.04.2019',
 '26.08.2019',
 '12.11.2018',
 '29.01.2019',
 '04.12.2018',
 '26.12.2019',
 '10.05.2019']

#### re.split()

Falls in dem regulären Ausdruck für eine Splitoperation Capture Groups enthalten sind, werden die Teilmatches in die Ausgabe mit aufgenommen. Das kann z. B. praktisch sein, um nichtvorhersehbare Trennelemente zu erhalten.

In [9]:
text = "asdf1234xyz567rie2987zieo98237aresri2109rzrio"

print(re.split(r'\d+', text))
print('vs.')
print(re.split(r'(\d+)', text))

['asdf', 'xyz', 'rie', 'zieo', 'aresri', 'rzrio']
vs.
['asdf', '1234', 'xyz', '567', 'rie', '2987', 'zieo', '98237', 'aresri', '2109', 'rzrio']


## Greedy und Lazy Quantoren

Quantoren in regulären Ausdrücken sind standardmäßig *greedy*. Das heißt, sie versuchen, so viele Iterationen wie möglich zu matchen. Betroffen davon sind die Quantoren `*`, `+` und `{m,n}`. Da dieses Verhalten nicht in allen Fällen erwünscht ist, gibt es von jedem Quantor eine *lazy* Variante. Das bedeutet, dass der lazy Quantor versucht, so wenige Iterationen wie möglich zu matchen. Gebildet werden die lazy Quantoren mit einem nachgestellten Fragezeichen, also `*?`, `+?` und `{m,n}?`.

Ein Beispel, wo greedy Matching nicht erwünscht ist, ist z. B. das Matchen von XML-Tags.

In [10]:
xml = "Ich bin <em>Text mit inline XML-Tags</em>. Wenn man mich greedy Matcht, wird <strong>ganz schöner Murks</strong> dabei rauskommen. Deswegen sollten <marquee>lazy Quantoren</marquee> verwendet werden!"

greedy = r"<.+>"

print("Greedy:")
print(re.findall(greedy, xml))

lazy = r"<.+?>"

print("\nLazy:")
print(re.findall(lazy, xml))

Greedy:
['<em>Text mit inline XML-Tags</em>. Wenn man mich greedy Matcht, wird <strong>ganz schöner Murks</strong> dabei rauskommen. Deswegen sollten <marquee>lazy Quantoren</marquee>']

Lazy:
['<em>', '</em>', '<strong>', '</strong>', '<marquee>', '</marquee>']


## Lookaround

Lookaround ist der Oberbegriff für Teilausdrücke, die eingelesen werden müssen, aber selbst nicht zum Match beitragen. Sprich, sie stellen Konditionen dar, die erfüllt werden müssen, damit ein Match erfolgen kann.

### Lookahead

Beim Lookahead beeinflusst das, was nach dem eigentlichen Muster steht, ob das Muster selbst gematcht wird. Dabei gibt es zwei Varianten: Positive und negative Lookahead.
- Der positive Lookahead beschreibt ein Muster, das auf das eigentlich zu matchende Muster folgen muss. Er wird mit der Syntax `(?=...)` angegeben. Beim Ausdruck `A(?=B)` wird "A" nur gematcht, wenn auf es ein "B" folgt.
- Der negative Lookahead beschreibt ein Muster, das nicht auf das eigentlich zu matchende Muster folgen darf. Er wird mit der Syntax `(?!...)` angegeben. Beim Ausdruck `A(?!B)` wird "A" nur gematcht, wenn auf es *kein* "B" folgt.

In [11]:
text = """Der Große Panda (Ailuropoda melanoleuca), auch Riesenpanda oder Pandabär, ist eine Säugetierart aus der Familie der Bären (Ursidae). Als Symbol des WWF und manchmal auch des Artenschutzes allgemein hat er trotz seines sehr beschränkten Verbreitungsgebiets weltweite Bekanntheit erlangt. In älterer deutscher Literatur wird der Große Panda auch Bambusbär oder Prankenbär genannt. Zurzeit gibt es schätzungsweise 1864 frei lebende Exemplare. Darunter ist mindestens ein Albino.
Große Pandas erreichen eine Kopfrumpflänge von 120 bis 150 Zentimetern, der Schwanz ist wie bei allen Bären nur ein Stummel von rund 12 Zentimetern Länge. Das Gewicht erwachsener Tiere variiert von 75 bis 160 Kilogramm. Große Pandas entsprechen in ihrem Körperbau weitgehend den anderen Bären, stechen jedoch durch ihre kontrastreiche schwarz-weiße Färbung hervor.
Die Grundfarbe ihres dichten, wolligen Fells ist weiß, die Beine sind schwarz. Das Schwarz der Vorderbeine zieht sich weiter über die Schultern und bildet einen Gürtel, der meist den Vorderkörper umschließt. Schwarz sind außerdem die Ohren, die Umgebung der Augen und manchmal die Schwanzspitze. Die Population im Qinling-Gebirge, die 2005 als erste offizielle Unterart anerkannt wurde, zeichnet sich dagegen durch eine braun weiße Farbgebung aus."""

# Diese Regex findet alle Wörter, die vor einem Satzzeichen stehen.
positive = r"\b\w+(?=[.,:;])"

# Diese Regex findet alle Satzzeichen, auf die kein Leerzeichen folgt.
negative = r"[.,:;](?!\s)"

print("Wörter vor Satzzeichen:", re.findall(positive, text))
print("\nSatzzeichen ohne folgendes Leerzeichen :", re.findall(negative, text))


Wörter vor Satzzeichen: ['Pandabär', 'erlangt', 'genannt', 'Exemplare', 'Albino', 'Zentimetern', 'Länge', 'Kilogramm', 'Bären', 'hervor', 'dichten', 'weiß', 'schwarz', 'Gürtel', 'umschließt', 'Ohren', 'Schwanzspitze', 'Gebirge', 'wurde', 'aus']

Satzzeichen ohne folgendes Leerzeichen : ['.']


### Lookbehind

Beim Lookbehind beeinflusst das, was vor dem eigentlichen Muster steht, ob das Muster selbst gematcht wird. Auch dabei gibt es positive und negative Lookbehind.
- Der positive Lookbehind beschreibt ein Muster, das vor dem eigentlich zu matchende Muster stehen muss. Er wird mit der Syntax `(?<=...)` angegeben. Beim Ausdruck `(?<=A)B` wird "B" nur gematcht, wenn es auf ein "A" folgt.
- Der negative Lookbehind beschreibt ein Muster, das nicht vor dem eigentlich zu matchenden Muster stehen darf. Er wird mit der Syntax `(?<!...)` angegeben. Beim Ausdruck `(?<!A)B` wird "B" nur gematcht, wenn es *nicht* auf ein "A" folgt.

Eine Besonderheit dabei ist, dass das Muster für den Lookbehind eine feste Länge haben muss. Es sind also keine Quantoren wie `+`, `*`, `?` und `{m,n}` erlaubt.

In [12]:
text = """Das Capybara oder Wasserschwein (Hydrochoerus hydrochaeris) ist eine Säugetierart aus der Familie der Meerschweinchen (Caviidae). Es bildet gemeinsam mit dem Panama-Capybara (Hydrochoerus isthmius) die Gattung Hydrochoerus und ist das größte heute lebende Nagetier. Es bewohnt feuchte Regionen in Südamerika und ist vom Körperbau seiner semiaquatischen (teilweise im Wasser stattfindenden) Lebensweise ideal angepasst.
Capybaras halten sich vorwiegend im Wasser auf. Die Schwimmhäute zwischen ihren Zehen helfen ihnen dabei, sich dort schnell zu bewegen. Ohren, Augen und Nase verlaufen in einer Linie im oberen Kopfbereich, ähnlich wie beim Kaiman. So können Capybaras fast mit dem gesamten Kopf unterhalb der Wasseroberfläche schwimmen und sind deswegen für etwaige Fressfeinde kaum zu erspähen. Das Geschlecht der Tiere ist nicht leicht zu bestimmen, da sich ihre Geschlechtsorgane im Körperinneren befinden und es keinen ausgeprägten Geschlechtsdimorphismus gibt. 
Das Capybara ist das größte heute lebende Nagetier. Es erreicht eine Kopf-Rumpf-Länge von 100 bis 134 Zentimetern sowie eine Schulterhöhe von 50 bis 62 Zentimetern, wobei die Weibchen etwas größer werden als die Männchen. Das Gewicht kann mehr als 75 Kilogramm betragen, das bekannte Maximalgewicht liegt bei 91 Kilogramm. Die Hinterfußlänge beträgt 21,8 bis 25,2 Zentimeter. Der Körper der Capybaras ist massiv und plump gebaut mit einem stämmigen Rumpf und kurzen Gliedmaßen. Die Vorderbeine enden in vier und die Hinterbeine in drei Zehen, die jeweils radial angeordnet sind. Die hufähnlich verdickten Zehen und Nägel sind durch kleine Schwimmhäute verbunden. Der Schwanz ist deutlich rückgebildet. Das Fell ist lang und rau, stellenweise aber so dünn, dass die Haut durchscheint. Seine Färbung variiert von rotbraun bis grau an der Oberseite, die Unterseite ist gelblich-braun gefärbt. Manche Tiere haben schwarze Flecken im Gesicht, an der Außenseite der Gliedmaßen und am Rumpf. Die Länge der Haare beträgt 30 bis 120 Millimeter."""

# Diese Regex findet allen Whitespace, der auf einen Punkt oder Doppelpunkt folgt.
positive = r"(?<=[.:])\s+"

# Diese Regex findet alle Wörter, die mit einem Großbuchstaben beginnnen, aber nicht auf ein Leerzeichen folgen
negative = r"(?<!\s)[A-ZÄÖÜ][a-zäöü]+"

sentences = re.split(positive, text)
print(sentences)

specialwords = re.findall(negative, text)
print(specialwords)

['Das Capybara oder Wasserschwein (Hydrochoerus hydrochaeris) ist eine Säugetierart aus der Familie der Meerschweinchen (Caviidae).', 'Es bildet gemeinsam mit dem Panama-Capybara (Hydrochoerus isthmius) die Gattung Hydrochoerus und ist das größte heute lebende Nagetier.', 'Es bewohnt feuchte Regionen in Südamerika und ist vom Körperbau seiner semiaquatischen (teilweise im Wasser stattfindenden) Lebensweise ideal angepasst.', 'Capybaras halten sich vorwiegend im Wasser auf.', 'Die Schwimmhäute zwischen ihren Zehen helfen ihnen dabei, sich dort schnell zu bewegen.', 'Ohren, Augen und Nase verlaufen in einer Linie im oberen Kopfbereich, ähnlich wie beim Kaiman.', 'So können Capybaras fast mit dem gesamten Kopf unterhalb der Wasseroberfläche schwimmen und sind deswegen für etwaige Fressfeinde kaum zu erspähen.', 'Das Geschlecht der Tiere ist nicht leicht zu bestimmen, da sich ihre Geschlechtsorgane im Körperinneren befinden und es keinen ausgeprägten Geschlechtsdimorphismus gibt.', 'Das Ca